# *Telecom X - Análisis de Evasión de Clientes*

*Has sido contratado como asistente de análisis de datos en Telecom X y formarás parte del proyecto "Churn de Clientes". La empresa enfrenta una alta tasa de cancelaciones y necesita comprender los factores que llevan a la pérdida de clientes.*

*Tu desafío será recopilar, procesar y analizar los datos, utilizando Python y sus principales bibliotecas para extraer información valiosa. A partir de tu análisis, el equipo de Data Science podrá avanzar en modelos predictivos y desarrollar estrategias para reducir la evasión.*

## Instalación de librerias

In [208]:
!!! pip install pandas
!!! pip install matplotlib
!!! pip install numpy

["'!' is not recognized as an internal or external command,",
 'operable program or batch file.']

## Importar librerias

In [209]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Extracción(E - Extract)

## Extracción de datos

*Para iniciar tu análisis, necesitarás importar los datos de la API de Telecom X. Estos datos están disponibles en formato JSON y contienen información esencial sobre los clientes, incluyendo datos demográficos, tipo de servicio contratado y estado de evasión.*

## Importar el dataset

In [210]:
datos = pd.read_json('json/TelecomX_Data.json')

## Visualizar el dataset

In [211]:
datos.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


# Transformación (T - Transform)

# Conoce el conjunto de datos

Ahora que has extraído los datos, es fundamental comprender la estructura del dataset y el significado de sus columnas. Esta etapa te ayudará a identificar qué variables son más relevantes para el análisis de evasión de clientes.

In [212]:
datos.info()

<class 'pandas.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   str   
 1   Churn       7267 non-null   str   
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(4), str(2)
memory usage: 340.8+ KB


In [213]:
datos.dtypes

customerID       str
Churn            str
customer      object
phone         object
internet      object
account       object
dtype: object

## Normalizar los diccionarios del JSON

In [214]:
customer_df = pd.json_normalize(datos['customer'])
phone_df = pd.json_normalize(datos['phone'])
internet_df = pd.json_normalize(datos['internet'])
account_df = pd.json_normalize(datos['account'])

## Unir todo en un solo DataFrame

In [215]:
df_flat = pd.concat(
    [
        datos[['customerID', 'Churn']],
        customer_df,
        phone_df,
        internet_df,
        account_df
    ],
    axis=1
)

In [216]:
df_flat.info()

<class 'pandas.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   str    
 1   Churn             7267 non-null   str    
 2   gender            7267 non-null   str    
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   str    
 5   Dependents        7267 non-null   str    
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   str    
 8   MultipleLines     7267 non-null   str    
 9   InternetService   7267 non-null   str    
 10  OnlineSecurity    7267 non-null   str    
 11  OnlineBackup      7267 non-null   str    
 12  DeviceProtection  7267 non-null   str    
 13  TechSupport       7267 non-null   str    
 14  StreamingTV       7267 non-null   str    
 15  StreamingMovies   7267 non-null   str    
 16  Contract          7267 non-null   str    
 17  Paperl

In [217]:
df_flat.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [218]:
use_columns = ['customerID', 'Churn', 'tenure','PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'Charges.Monthly', 'Charges.Total', 'PaymentMethod']

In [219]:
df_use = df_flat[use_columns]
df_use.head()

,customerID,Churn,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,Charges.Monthly,Charges.Total,PaymentMethod
0,0002-ORFBO,No,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,65.6,593.3,Mailed check
1,0003-MKNFE,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,59.9,542.4,Mailed check
2,0004-TLHLJ,Yes,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,73.9,280.85,Electronic check
3,0011-IGKFF,Yes,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,98.0,1237.85,Electronic check
4,0013-EXCHZ,Yes,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,83.9,267.4,Mailed check


In [220]:
df_use.tail()

,customerID,Churn,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,Charges.Monthly,Charges.Total,PaymentMethod
7262,9987-LUTYD,No,13,Yes,No,DSL,Yes,No,No,Yes,No,No,One year,55.15,742.9,Mailed check
7263,9992-RRAMN,Yes,22,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,85.10,1873.7,Electronic check
7264,9992-UJOEL,No,2,Yes,No,DSL,No,Yes,No,No,No,No,Month-to-month,50.30,92.75,Mailed check
7265,9993-LHIEB,No,67,Yes,No,DSL,Yes,No,Yes,Yes,No,Yes,Two year,67.85,4627.65,Mailed check
7266,9995-HOTOH,No,63,No,No phone service,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,59.00,3707.6,Electronic check


In [221]:
df_use.info()

<class 'pandas.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   str    
 1   Churn             7267 non-null   str    
 2   tenure            7267 non-null   int64  
 3   PhoneService      7267 non-null   str    
 4   MultipleLines     7267 non-null   str    
 5   InternetService   7267 non-null   str    
 6   OnlineSecurity    7267 non-null   str    
 7   OnlineBackup      7267 non-null   str    
 8   DeviceProtection  7267 non-null   str    
 9   TechSupport       7267 non-null   str    
 10  StreamingTV       7267 non-null   str    
 11  StreamingMovies   7267 non-null   str    
 12  Contract          7267 non-null   str    
 13  Charges.Monthly   7267 non-null   float64
 14  Charges.Total     7267 non-null   str    
 15  PaymentMethod     7267 non-null   str    
dtypes: float64(1), int64(1), str(14)
memory usage: 908.5 

In [222]:
df_use.dtypes

customerID              str
Churn                   str
tenure                int64
PhoneService            str
MultipleLines           str
InternetService         str
OnlineSecurity          str
OnlineBackup            str
DeviceProtection        str
TechSupport             str
StreamingTV             str
StreamingMovies         str
Contract                str
Charges.Monthly     float64
Charges.Total           str
PaymentMethod           str
dtype: object

In [223]:
df_use['Charges.Total'] = df_use['Charges.Total'].replace(' ', np.nan)

In [224]:
df_use['Charges.Total'] = df_use['Charges.Total'].astype(np.float64)

In [225]:
df_use.dtypes

customerID              str
Churn                   str
tenure                int64
PhoneService            str
MultipleLines           str
InternetService         str
OnlineSecurity          str
OnlineBackup            str
DeviceProtection        str
TechSupport             str
StreamingTV             str
StreamingMovies         str
Contract                str
Charges.Monthly     float64
Charges.Total       float64
PaymentMethod           str
dtype: object

# Comprobación de incoherencias en los datos

*En este paso, verifica si hay problemas en los datos que puedan afectar el análisis. Presta atención a valores ausentes, duplicados, errores de formato e inconsistencias en las categorías. Este proceso es esencial para asegurarte de que los datos estén listos para las siguientes etapas.*

In [226]:
df_use.isnull()

,customerID,Churn,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,Charges.Monthly,Charges.Total,PaymentMethod
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7263,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7264,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7265,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [227]:
df_use.isna().sum()

customerID           0
Churn                0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
Charges.Monthly      0
Charges.Total       11
PaymentMethod        0
dtype: int64

In [233]:
df_use = df_use.fillna(0)
df_use.head()

,customerID,Churn,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,Charges.Monthly,Charges.Total,PaymentMethod
0,0002-ORFBO,No,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,65.6,593.30,Mailed check
1,0003-MKNFE,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,59.9,542.40,Mailed check
2,0004-TLHLJ,Yes,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,73.9,280.85,Electronic check
3,0011-IGKFF,Yes,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,98.0,1237.85,Electronic check
4,0013-EXCHZ,Yes,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,83.9,267.40,Mailed check


In [229]:
df_use.isna().sum()

customerID          0
Churn               0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
Charges.Monthly     0
Charges.Total       0
PaymentMethod       0
dtype: int64

In [230]:
df_use.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
7262    False
7263    False
7264    False
7265    False
7266    False
Length: 7267, dtype: bool

In [231]:
df_use[df_use.duplicated()]

,customerID,Churn,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,Charges.Monthly,Charges.Total,PaymentMethod


In [232]:
df_use.duplicated().sum()

np.int64(0)

# Manejo de inconsistencias

*Ahora que has identificado las inconsistencias, es momento de aplicar las correcciones necesarias. Ajusta los datos para asegurarte de que estén completos y coherentes, preparándolos para las siguientes etapas del análisis.*